0 导入包

In [ ]:
'''
必要加载项  model tokenizer feature
Processor = tokenizer特征标记器 + feature_extractor特征放大器

'''


In [1]:
# 处理数据集
from datasets import load_dataset
from datasets import Audio

# 加载模型
from transformers import pipeline
from transformers import AutoProcessor, AutoModel, MusicgenForConditionalGeneration

# 播放音乐
from IPython.display import Audio as AU # 接收(array，int)，输出音频

# 保存音频
import time # 用于命名
import soundfile as sf # 保存
import numpy as np  # 数据格式转换

# 调用gpu
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
# 导入数据集
minds = load_dataset("PolyAI/minds14", name="en-AU", split="train") # 加载数据集
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))  # 更改数据集默认采样率

1 输入为音频

1.1 vocie -> tag

In [11]:
# ——————————pipeline加载——————————
classifier = pipeline("audio-classification",model="anton-l/xtreme_s_xlsr_300m_minds14",device=0,)
classifier(minds[0]["audio"]["array"]) # 输入音频numpy数组，或文件地址，输出预测结果

Some weights of the model checkpoint at anton-l/xtreme_s_xlsr_300m_minds14 were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at anton-l/xtreme_s_xlsr_300m_minds14 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos

[{'score': 0.9625251889228821, 'label': 'pay_bill'},
 {'score': 0.028678132221102715, 'label': 'freeze'},
 {'score': 0.003350093262270093, 'label': 'card_issues'},
 {'score': 0.002005866961553693, 'label': 'abroad'},
 {'score': 0.0008484389400109649, 'label': 'high_value_payment'}]

1.2 voice -> text

In [12]:
# ——————————pipeline加载——————————
asr = pipeline("automatic-speech-recognition",model = "facebook/wav2vec2-base-960h",device=0,)
asr(minds[0]["audio"]["array"]) # 输入音频numpy数组，或文件地址，输出预测结果

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

{'text': 'I WOULD LIKE TO PAY MY ELECTRICITY BILL USING MY CAD CAN YOU PLEASE ASSIST'}

2 输入为文本

2.1 text -> voice

In [2]:
# ——————————pipeline加载——————————
pipe = pipeline("text-to-speech", model="suno/bark-small",device=0,)
text = "Ladybugs have had important roles in culture and religion, being associated with luck, love, fertility and prophecy. "
output = pipe(text)
print(output["audio"].shape)
AU(output["audio"], rate=output["sampling_rate"])

d:\env_config\py\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
d:\env_config\py\lib\site-packages\transformers\models\encodec\modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


(1, 296000)


In [14]:
# ——————————transformers加载——————————
processor = AutoProcessor.from_pretrained("suno/bark-small")
model = AutoModel.from_pretrained("suno/bark-small").to(device).half() # 转移到gpu，半精度加载

# 输入与模型需要在同一台设备上
inputs = processor(
    text = ["Ladybugs have had important roles in culture and religion, being associated with luck, love, fertility and prophecy. "],
    return_tensors="pt",
).to(device)

speech_values = model.generate(**inputs, do_sample=True) # torch gpu (1,n) f16 生成
speech_values = speech_values[0].cpu().numpy().astype(np.float32) # numpy cpu (n) f32 转换数据格式
sampling_rate = model.generation_config.sample_rate # 获取采样率
#
AU(speech_values, rate=sampling_rate) # 播放

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


2.2 text -> song

In [15]:
# ——————————pipeline加载——————————
pipe = pipeline("text-to-speech", model="suno/bark-small",device=0,)
song = "♪ In the jungle, the mighty jungle, the ladybug was seen. ♪ "
output = pipe(song)
AU(output["audio"], rate=output["sampling_rate"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [16]:
# ——————————transformers加载——————————
processor = AutoProcessor.from_pretrained("suno/bark-small")
model = AutoModel.from_pretrained("suno/bark-small").to(device).half() # 转移到gpu，半精度加载

In [17]:
# 输入与模型需要在同一台设备上
inputs = processor(
    text=["♪ In the jungle, the mighty jungle, the ladybug was seen. ♪ "],
    return_tensors="pt",
).to(device)

speech_values = model.generate(**inputs, do_sample=True) # torch gpu (1,n) f16 生成
speech_values = speech_values[0].cpu().numpy().astype(np.float32) # numpy cpu (n) f32 转换数据格式
sampling_rate = model.generation_config.sample_rate # 获取采样率

AU(speech_values, rate=sampling_rate) # 播放

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


3 输入为prompt

In [18]:
# ——————————transformers加载——————————
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").to(device).half() # 转移到gpu，半精度加载

In [19]:
# prompt -> music
# 输入与模型需要在同一台设备上
inputs = processor(
    text = ["lo-fi music with a soothing melody"], # 可输入str或列表
    padding=True,
    return_tensors="pt",
).to(device)

audio_values = model.generate(**inputs, max_new_tokens=512) # torch gpu (1,1,n) f16 生成音乐
audio_values = audio_values[0,0].cpu().numpy().astype(np.float32) # numpy cpu (n) f32 转换数据格式
sampling_rate = model.config.audio_encoder.sampling_rate # 获取采样率

AU(audio_values, rate=sampling_rate) # 播放

In [ ]:
# 保存
sf.write(f"{int(time.time())}.wav", audio_values.T, 44100, 'PCM_24')